In [8]:
# region General Imports
import os
import re
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

import tempfile
import tensorflow as tf
import numpy as np
from tensorflow import keras

import tensorflow_model_optimization as tfmot
%load_ext tensorboard
    
# os.environ["KERAS_BACKEND"] = "tf"
# os.environ["TF_USE_LEGACY_KERAS"] = "0"
# import jax
# import jax.numpy as jnp
# import keras
    
# region Keras
from keras.models import Model
from keras.layers import Dense, Activation, Permute, Dropout
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    Conv1D,
    MaxPooling1D,
    AveragePooling1D,
)
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D
from keras.regularizers import l1_l2
from keras.layers import Input, Flatten
from keras.constraints import max_norm
from keras import backend as K
# endregion Keras

from custom_datasets.fatigue_mi import FatigueMI

# Sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, BatchNormalization
from sklearn import preprocessing

from model_optim.utils import channels_to_channels_idx

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
SKLRNG = 42
# RNG = jax.random.PRNGKey(SKLRNG)

In [10]:
# region Helper funcs
def shallow_conv_net_square_layer(x):
    return tf.math.square(x)

def shallow_conv_net_log_layer(x):
    return tf.math.log(tf.clip_by_value(x, 1e-7, 10000))

CUSTOM_OBJECTS = {
    "shallow_conv_net_square_layer": shallow_conv_net_square_layer, 
    "shallow_conv_net_log_layer": shallow_conv_net_log_layer 
}
# endregion Helper funcs

# region Models
def shallow_conv_net(
    nb_classes, channels, samples, **kwargs
):
    """
    From: https://github.com/vlawhern/arl-eegmodels/blob/master/EEGModels.py
    """

    _POOL_SIZE_D2_ = kwargs.get("pool_size_d2", 35)
    _STRIDES_D2_ = kwargs.get("strides_d2", 7)
    _CONV_FILTERS_D2_ = kwargs.get("conv_filters_d2", 13)

    _POOL_SIZE_ = kwargs.get("pool_size", (1, _POOL_SIZE_D2_))
    _STRIDES_ = kwargs.get("strides", (1, _STRIDES_D2_))
    _CONV_FILTERS_ = kwargs.get("conv_filters", (1, _CONV_FILTERS_D2_))

    _CONV2D_1_UNITS_ = kwargs.get("conv2d_1_units", 40)
    _CONV2D_2_UNITS_ = kwargs.get("conv2d_2_units", 40)
    _L2_REG_1_ = kwargs.get("l2_reg_1", 0.01)
    _L2_REG_2_ = kwargs.get("l2_reg_2", 0.01)
    _L2_REG_3_ = kwargs.get("l2_reg_3", 0.01)
    _DROPOUT_RATE_ = kwargs.get("dropout_rate", 0.5)

    input_main = Input(shape=(channels, samples, 1))
    block1 = Conv2D(
        _CONV2D_1_UNITS_,
        _CONV_FILTERS_,
        input_shape=(channels, samples, 1),
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_1_),
    )(input_main)
    # block1       = Conv2D(40, (channels, 1), use_bias=False,
    #                       kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1 = Conv2D(
        _CONV2D_2_UNITS_,
        (channels, 1),
        use_bias=False,
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_2_),
    )(block1)
    block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1 = Activation(shallow_conv_net_square_layer)(block1)
    block1 = AveragePooling2D(pool_size=_POOL_SIZE_, strides=_STRIDES_)(block1)
    block1 = Activation(shallow_conv_net_log_layer)(block1)
    block1 = Dropout(_DROPOUT_RATE_)(block1)
    flatten = Flatten()(block1)
    # dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    dense = Dense(
        nb_classes,
        kernel_constraint=max_norm(0.5),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_3_),
    )(flatten)
    softmax = Activation("softmax")(dense)

    return Model(inputs=input_main, outputs=softmax)

# endregion Models

In [11]:
# subject_best_trials = glob.glob('./temp_v2/**/model/study_best_trial.npy', recursive=True)
# subject_best_trials = sorted(subject_best_trials, key=lambda x: os.path.getmtime(x))
# subject_best_trials = sorted(subject_best_trials, key=lambda x: int(re.compile(r"\[.*\]").search(x).group(0).strip("[]")))

subject_best_trials = glob.glob('./temp/**/model/study_best_trial.npy', recursive=True)

In [12]:
def data_generator(dataset, subjects = [1], channel_idx = [], filters = ([8, 32],), sfreq = 250):

    find_events = lambda raw, event_id: mne.find_events(raw, shortest_event=0, verbose=False) if len(mne.utils._get_stim_channel(None, raw.info, raise_error=False)) > 0 else mne.events_from_annotations(raw, event_id=event_id, verbose=False)[0]
    
    data = dataset.get_data(subjects=subjects)
    
    X = []
    y = []
    metadata = []

    for subject_id in data.keys():
        for session_id in data[subject_id].keys():
            for run_id in data[subject_id][session_id].keys():
                raw = data[subject_id][session_id][run_id]
                
                for fmin, fmax in filters:
                    raw = raw.filter(l_freq = fmin, h_freq = fmax, method = 'iir', picks = 'eeg', verbose = False)
                
                events = find_events(raw, dataset.event_id)

                tmin = dataset.interval[0]
                tmax = dataset.interval[1]

                channels = np.asarray(raw.info['ch_names'])[channel_idx] if len(channel_idx) > 0 else np.asarray(raw.info['ch_names'])

                # rpprint(channels)
                
                stim_channels = mne.utils._get_stim_channel(None, raw.info, raise_error=False)
                picks = mne.pick_channels(raw.info["ch_names"], include=channels, exclude=stim_channels, ordered=True)

                x = mne.Epochs(
                    raw,
                    events,
                    event_id=dataset.event_id,
                    tmin=tmin,
                    tmax=tmax,
                    proj=False,
                    baseline=None,
                    preload=True,
                    verbose=False,
                    picks=picks,
                    event_repeated="drop",
                    on_missing="ignore",
                )
                x_events = x.events
                inv_events = {k: v for v, k in dataset.event_id.items()}
                labels = [inv_events[e] for e in x_events[:, -1]]

                # rpprint({
                #     "X": np.asarray(x.get_data(copy=False)).shape,
                #     "y": np.asarray(labels).shape,
                #     "channels selected": np.asarray(raw.info['ch_names'])[channel_idx]
                # })

                # x.plot(scalings="auto")
                # display(x.info)
                
                x_resampled = x.resample(sfreq) # Resampler_Epoch
                x_resampled_data = x_resampled.get_data(copy=False) # Convert_Epoch_Array
                x_resampled_data_standard_scaler = np.asarray([
                    StandardScaler().fit_transform(x_resampled_data[i])
                    for i in np.arange(x_resampled_data.shape[0])
                ]) # Standard_Scaler_Epoch

                # x_resampled.plot(scalings="auto")
                # display(x_resampled.info)

                n = x_resampled_data_standard_scaler.shape[0]
                # n = x.get_data(copy=False).shape[0]
                met = pd.DataFrame(index=range(n))
                met["subject"] = subject_id
                met["session"] = session_id
                met["run"] = run_id
                x.metadata = met.copy()
                
                # X.append(x_resampled_data_standard_scaler)
                X.append(x)
                y.append(labels)
                metadata.append(met)

    return np.concatenate(X, axis=0), np.concatenate(y), pd.concat(metadata, ignore_index=True)

fat_dataset = FatigueMI()

In [13]:
rpprint(subject_best_trials)

['./temp/[1]/8ca44c9b9c7c4410b37ac5781bd7da1f/model/study_best_trial.npy']

In [14]:
def load_best_trial(subject_best_trial):
    model = np.load(subject_best_trials[0], allow_pickle=True).item()
    model_info = {
        "subject": model.user_attrs["trial_data"]["subject"] if hasattr(model.user_attrs["trial_data"], "subject") else int(re.compile(r"\[.*\]").search(model.user_attrs["trial_data"]["data_path"]).group(0).strip("[]")),
        "sfreq": model.params["sfreq"] if "sfreq" in model.params else 128,
        "batch_size": model.params["batch_size"] if "batch_size" in model.params else 128,
        "channels_selected": model.user_attrs["trial_data"]["channels_selected"],
        "channels_idx_selected": channels_to_channels_idx(model.user_attrs["trial_data"]["channels_selected"], fat_dataset.get_data(subjects=[1])[1]['0']['0'].info['ch_names'][:-1]),
        "model": tf.keras.models.model_from_json(model.user_attrs["trial_data"]["model"], custom_objects=CUSTOM_OBJECTS),
        "test_acc": model.user_attrs["trial_data"]["test_accuracy"],
        "model_name": model.user_attrs["trial_data"]["model_name"] if hasattr(model.user_attrs["trial_data"], "model_name") else "shallow_conv_net"
    }
    if "weights" in model.user_attrs["trial_data"]:
        model_info["model"].set_weights(model.user_attrs["trial_data"]["weights"])
    elif "model_weights" in model.user_attrs["trial_data"]:
        model_info["model"].set_weights(model.user_attrs["trial_data"]["model_weights"])
    
    return model_info

def create_and_save_baseline_model(model_info, train_test_data, results_folder):

    X_test, y_test = train_test_data["X_test"], train_test_data["y_test"]

    model_info["model"].compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    baseline_model_test = model_info["model"].evaluate(X_test, y_test, batch_size=model_info["batch_size"])

    baseline_test_acc = baseline_model_test[1]

    _, keras_file = None, results_folder + "baseline_model.h5"
    tf.keras.models.save_model(model_info["model"], keras_file, include_optimizer=False)
    return {
        "keras_file": keras_file,
        "baseline_test_acc": baseline_test_acc,
        "baseline_model": model_info["model"],
    }

def prune_model(model_info, train_test_data, target_sparsity, results_folder):

    X_train, y_train = train_test_data["X_train"], train_test_data["y_train"]
    X_test, y_test = train_test_data["X_test"], train_test_data["y_test"]

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    batch_size = model_info["batch_size"]
    epochs = 2
    end_step = np.ceil(len(X_train) / batch_size).astype(np.int32) * epochs
    # Define model for pruning.
    pruning_params = {
        # 'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=target_sparsity, begin_step=0, frequency=1),
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.00,
                                                                final_sparsity=target_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step,
                                                                frequency=1)
    }
    keras.utils.get_custom_objects().update({
        **CUSTOM_OBJECTS
    })
    baseline_model_copy = tf.keras.models.clone_model(model_info["model"])
    model_for_pruning = prune_low_magnitude(baseline_model_copy, **pruning_params)
    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    logdir = "logs/pruning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    model_for_pruning.fit(X_train, y_train, batch_size=model_info["batch_size"], epochs=epochs, validation_split=0.2, callbacks=callbacks)
    _, model_for_pruning_accuracy = model_for_pruning.evaluate(X_test, y_test, verbose=0)

    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
    _, pruned_keras_file = None, results_folder + "pruned_model.h5"
    tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

    return {
        "pruned_model": model_for_export,
        "pruned_model_test_acc": model_for_pruning_accuracy,
        "pruned_keras_file": pruned_keras_file
    }

def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    import os
    import zipfile
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    
    return os.path.getsize(zipped_file)

def convert_pruned_model_to_tflite(pruned_model, sparsity, results_folder):
    converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.target_spec.supported_types = [quantization]
    pruned_tflite_model = converter.convert()

    _, pruned_tflite_file = None, results_folder + f"pruned_model_{sparsity}_sparsity.tflite"
    with open(pruned_tflite_file, 'wb') as f:
        f.write(pruned_tflite_model)
    
    return {
        "pruned_tflite_model": pruned_tflite_model,
        "pruned_tflite_file": pruned_tflite_file
    }

def convert_pruned_model_to_tflite_with_quantization(pruned_model, sparsity, quantization, results_folder):
    converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    if quantization == 'float16':
        converter.target_spec.supported_types = [tf.float16]
    elif quantization == 'int8':
        pass
    else:
        raise ValueError(f"Quantization type {quantization} not implemented.")
    pruned_tflite_model = converter.convert()

    _, pruned_tflite_file = None, results_folder + f"pruned_model_{sparsity}_sparsity_{quantization}_quant.tflite"
    with open(pruned_tflite_file, 'wb') as f:
        f.write(pruned_tflite_model)
    
    return {
        "pruned_quant_tflite_model": pruned_tflite_model,
        "pruned_quant_tflite_file": pruned_tflite_file
    }

def get_test_acc_non_tf_lite(model, train_test_data):

    X_test, y_test = train_test_data["X_test"], train_test_data["y_test"]

    # Evaluate prediction accuracy of pruned model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=0)

    # Evaluate Inference Time of pruned model
    start_time = time.time()
    prediction = model.predict(X_test)
    exec_time = (time.time() - start_time)/X_test.shape[0]
    return {
        "test_accuracy": test_acc,
        "avg_exec_time": exec_time
    }

def get_test_acc(model, train_test_data):
    X_test, y_test = train_test_data["X_test"], train_test_data["y_test"]
    def evaluate_model(interpreter):
        input_index = interpreter.get_input_details()[0]["index"]
        output_index = interpreter.get_output_details()[0]["index"]

        # signatures = interpreter.get_signature_list()
        # rprint(interpreter.get_input_details(), interpreter.get_output_details(), signatures)

        # Run predictions on every image in the "test" dataset.
        predictions = []
        exec_times = []
        for i, v in enumerate(X_test):
            v = v[np.newaxis, :, :, np.newaxis].astype(np.float32)
            # if i % 1000 == 0:
            #   rprint('Evaluated on {n} results so far.'.format(n=i))
            # # Pre-processing: add batch dimension and convert to float32 to match with
            # # the model's input data format.
            # v = np.expand_dims(v, axis=0).astype(np.float32)
            interpreter.set_tensor(input_index, v)

            # Run inference.
            start_time = time.time()
            interpreter.invoke()
            exec_time = (time.time() - start_time)
            exec_times.append(exec_time)

            # Post-processing: remove batch dimension and find the digit with highest
            # probability.
            output = interpreter.tensor(output_index)
            class_prediction = np.argmax(output()[0]) # 0 = left, 1 = right
            predictions.append(class_prediction)

        print('\n')
        # Compare prediction results with ground truth labels to calculate accuracy.
        predictions = np.asarray(predictions)
        accuracy = (predictions == y_test).mean()
        avg_exec_time = np.mean(exec_times)
        return accuracy, avg_exec_time

    interpreter = tf.lite.Interpreter(model_content=model)
    interpreter.allocate_tensors()

    test_accuracy, avg_exec_time = evaluate_model(interpreter)

    return {
        "test_accuracy": test_accuracy,
        "avg_exec_time": avg_exec_time
    }

def get_model_weights_sparsity(model):
    sparsity_levels = []
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights

        for weight in weights:
            # if "kernel" not in weight.name or "centroid" in weight.name:
            #     continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            sparsity_levels.append((
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            ))
    return sparsity_levels

In [15]:
# Folder structure for results: ./results/{subject}/{model}/{sparsity}/{quantization}/

sparsity_levels = [0, 0.1, 0.5, 0.9]
quantization_levels = [None, 'float16', 'int8']

results_data = {
    "subject": [],
    "model_name": [],
    "sparsity": [],
    "quantization": [],
    "test_acc": [],
    "model_size": [],
    "inf_time": [],
    "weights_sparsity": [],
}

for subject_best_trial in subject_best_trials[0:1]:

    rprint("\n\n\nLoading best trial model...", subject_best_trial)

    model_info = load_best_trial(subject_best_trial)
    subject = model_info["subject"]
    model_name = model_info["model_name"]

    X, y, _ = data_generator(fat_dataset, subjects=[model_info["subject"]], channel_idx=model_info["channels_idx_selected"], sfreq=model_info["sfreq"])
    y_encoded = LabelEncoder().fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=SKLRNG, shuffle=True, stratify=y_encoded)
    train_test_data = { "X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test }

    for sparsity in sparsity_levels:
        for quantization in quantization_levels:

            rprint(f"\n\n\nSubject: {subject}, Model: {model_name}, Sparsity: {sparsity}, Quantization: {quantization}\n\n")

            results_folder = f"./results/{subject}/{model_name}/{sparsity}/{quantization}/"
            os.makedirs(results_folder, exist_ok=True) if not os.path.exists(results_folder) else None # Make folder if doesn't exist

            # region 1) ------------- Build & save baseline model ------------- 
            rprint("Building & saving baseline model...")
            baseline_model_info = create_and_save_baseline_model(**{
                "model_info": model_info,
                "results_folder": results_folder,
                "train_test_data": train_test_data
            })
            keras_file, baseline_test_acc = baseline_model_info["keras_file"], baseline_model_info["baseline_test_acc"]
            rprint('Saved baseline model to:', keras_file)
            # endregion

            # region 2) ------------- Pruning the baseline model -------------
            rprint("Pruning the baseline model...")
            pruned_model_info = prune_model(**{
                "target_sparsity": sparsity,
                "model_info": model_info,
                "train_test_data": train_test_data,
                "results_folder": results_folder
            })
            
            pruned_model = pruned_model_info["pruned_model"]
            pruned_model_test_acc, pruned_keras_file = pruned_model_info["pruned_model_test_acc"], pruned_model_info["pruned_keras_file"]
            pruned_model_weights_sparsity = get_model_weights_sparsity(pruned_model)
            model_weights_sparsity = pruned_model_weights_sparsity

            rprint(f'Saved pruned Keras model with {sparsity*100}% sparsity to:', pruned_keras_file)
            rpprint({
                'Baseline test accuracy': baseline_test_acc,
                'Pruned test accuracy': pruned_model_test_acc
            })
            # endregion

            # region 3) ------------- Converting pruned model to TFLite -------------        
            if quantization != None:
                tflite_model_info = convert_pruned_model_to_tflite(**{
                    "pruned_model": pruned_model,
                    "sparsity": sparsity,
                    "results_folder": results_folder
                })
                pruned_tflite_model, pruned_tflite_file = tflite_model_info["pruned_tflite_model"], tflite_model_info["pruned_tflite_file"]
                
                rprint('Saved pruned TFLite model to:', pruned_tflite_file)
                rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
                rprint("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
                rprint("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))
            else:
                pruned_tflite_model, pruned_tflite_file = None, None
            # endregion

            # region 4) ------------- Quantizing pruned TFLite model ------------- 
            if quantization != None:
                tflite_quant_model_info = convert_pruned_model_to_tflite_with_quantization(**{
                    "pruned_model": pruned_model,
                    "sparsity": sparsity,
                    "quantization": quantization,
                    "results_folder": results_folder
                })
                quantized_and_pruned_tflite_model = tflite_quant_model_info["pruned_quant_tflite_model"]
                quantized_and_pruned_tflite_file = tflite_quant_model_info["pruned_quant_tflite_file"]

                rprint('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)
                rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
                rprint("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
            else:
                quantized_and_pruned_tflite_model, quantized_and_pruned_tflite_file = None, None
            
            # endregion

            # region 5) ------------- Evaluation check -------------
            if quantization != None:
                tflite_quant_model_test_acc_results = get_test_acc(**{
                    "model": quantized_and_pruned_tflite_model, 
                    "train_test_data": train_test_data
                })
                tflite_quant_model_test_acc = tflite_quant_model_test_acc_results["test_accuracy"]

                rprint('Pruned and quantized TFLite test_accuracy:', tflite_quant_model_test_acc)
                rprint('Pruned TF test accuracy:', pruned_model_test_acc)
            # endregion
            
            # region 6) ------------- Saving results -------------
            if quantization != None:
                model_objs = [{ "model": quantized_and_pruned_tflite_model, "file": quantized_and_pruned_tflite_file, "tf_lite": True, "weights_sparsity": model_weights_sparsity }]
            else:
                model_objs = [{ "model": pruned_model, "file": pruned_keras_file, "tf_lite": False, "weights_sparsity": model_weights_sparsity }]
                
            for model_obj in model_objs:
                model_to_evaluate = model_obj["model"]
                model_file_to_evaluate = model_obj["file"]
                model_is_tf_lite = model_obj["tf_lite"]
                model_weights_sparsity = model_obj["weights_sparsity"]

                if model_to_evaluate == None or model_file_to_evaluate == None:
                    continue

                # 1) Test accuracy & inference time
                if model_is_tf_lite:
                    eval_res = get_test_acc(**{
                        "model": model_to_evaluate, 
                        "train_test_data": train_test_data
                    })
                else:
                    eval_res = get_test_acc_non_tf_lite(**{
                        "model": model_to_evaluate, 
                        "train_test_data": train_test_data
                    })
                test_acc = eval_res["test_accuracy"]
                inf_time = eval_res["avg_exec_time"]
                # 2) Model size
                model_size = get_gzipped_model_size(model_file_to_evaluate)

                results_data["subject"].append(subject)
                results_data["model_name"].append(model_name)
                results_data["sparsity"].append(sparsity)
                results_data["quantization"].append(quantization)
                
                results_data["test_acc"].append(test_acc)
                results_data["model_size"].append(model_size)
                results_data["inf_time"].append(inf_time)

                results_data["weights_sparsity"].append(model_weights_sparsity)
            # endregion
    

Loading best trial model... ./temp/[1]/8ca44c9b9c7c4410b37ac5781bd7da1f/model/study_best_trial.npy

2024-04-01 19:08:56.603869: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 19:08:56.631927: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 19:08:56.632001: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 19:08:56.634999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 19:08:56.635207: I external/local_xla/xla/stream_executor

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns


Subject: 1, Model: shallow_conv_net, Sparsity: 0, Quantization: None

Building & saving baseline model...

...

Saved quantized and pruned TFLite model to: 
./results/1/shallow_conv_net/0.9/int8/pruned_model_0.9_sparsity_int8_quant.tflite

Size of gzipped baseline Keras model: 86070.00 bytes

Size of gzipped pruned and quantized TFlite model: 6962.00 bytes

Pruned and quantized TFLite test_accuracy: 0.5

Pruned TF test accuracy: 0.5

In [16]:
results_data_df = pd.DataFrame(results_data)
results_data_df

,subject,model_name,sparsity,quantization,test_acc,model_size,inf_time,weights_sparsity
0,1,shallow_conv_net,0.0,None,0.590909,85759,0.020158,"[(conv2d_20/kernel:0: 0.00% sparsity , (0/1610..."
1,1,shallow_conv_net,0.0,float16,0.363636,42636,0.000948,"[(conv2d_20/kernel:0: 0.00% sparsity , (0/1610..."
2,1,shallow_conv_net,0.0,int8,0.363636,24873,0.002454,"[(conv2d_20/kernel:0: 0.00% sparsity , (0/1610..."
3,1,shallow_conv_net,0.1,None,0.636364,80609,0.013621,"[(conv2d_20/kernel:0: 10.00% sparsity , (161/1..."
4,1,shallow_conv_net,0.1,float16,0.409091,41054,0.001223,"[(conv2d_20/kernel:0: 10.00% sparsity , (161/1..."
5,1,shallow_conv_net,0.1,int8,0.545455,23819,0.003124,"[(conv2d_20/kernel:0: 10.00% sparsity , (161/1..."
6,1,shallow_conv_net,0.5,None,0.500000,53333,0.013633,"[(conv2d_20/kernel:0: 50.00% sparsity , (805/1..."
7,1,shallow_conv_net,0.5,float16,0.318182,29079,0.001356,"[(conv2d_20/kernel:0: 50.00% sparsity , (805/1..."
8,1,shallow_conv_net,0.5,int8,0.590909,17574,0.004283,"[(conv2d_20/kernel:0: 50.00% sparsity , (805/1..."
9,1,shallow_conv_net,0.9,None,0.545455,20573,0.014336,"[(conv2d_20/kernel:0: 90.00% sparsity , (1449/..."
